# Strands Agents 빠른 시작 가이드

Strands Agents는 AWS 서비스와 상호작용하고 복잡한 작업을 수행할 수 있는 AI 에이전트를 구축하기 위한 강력한 프레임워크입니다. 이 빠른 시작 가이드는 첫 번째 Strands 에이전트를 만드는 데 도움을 줄 것입니다.

## 사전 요구사항

- Python 3.10 이상
- 적절한 권한으로 구성된 AWS 계정
- Python 프로그래밍에 대한 기본 이해

시작해봅시다!

In [ ]:
# pip를 사용하여 Strands 설치

!pip install strands-agents strands-agents-tools


## 첫 번째 에이전트 만들기

필요한 에이전트 구성 요소에 대한 개요를 살펴보겠습니다.

### 1. 간단한 에이전트 만들기:

이것은 기본 모델 제공자인 [Amazon Bedrock](https://aws.amazon.com/bedrock/)과 기본 모델인 Claude 3.7 Sonnet을 AWS 설정 지역에서 사용하는 에이전트를 만듭니다. 에이전트는 호출되는 것과 동일한 로컬 환경에서 실행되지만, Amazon Bedrock 모델은 AWS 계정에서 실행되고 에이전트는 클라우드 계정의 모델을 호출합니다. 아키텍처는 다음과 같습니다:

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent
# 에이전트 초기화
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    system_prompt="당신은 간결한 응답을 제공하는 도움이 되는 어시스턴트입니다."
)

# 에이전트에게 메시지 보내기
response = agent("오늘 strands agent workshop이 재미있을까?")

### 2. 에이전트에 도구 추가하기:

[strands-agents-tools](https://github.com/strands-agents/tools) 저장소는 가져올 수 있는 내장 도구들을 제공합니다. `@tool` 데코레이터를 사용하여 사용자 정의 도구를 만들 수도 있습니다. 내장 도구와 사용자 정의 도구를 가진 에이전트를 만들 수 있습니다. 예를 들어, 계산기의 내장 도구와 날씨를 가져오는 사용자 정의 도구를 추가하면 다음과 같은 아키텍처를 얻습니다:
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

이 아키텍처를 구현하면 다음과 같습니다:

In [ ]:
from strands import Agent, tool
from strands_tools import calculator # 계산기 도구 가져오기

# 사용자 정의 도구 만들기
@tool
def weather():
    """ 날씨 가져오기 """ # 더미 구현
    return "맑음"

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    tools=[calculator, weather],
    system_prompt="당신은 도움이 되는 어시스턴트입니다. 간단한 수학 계산을 할 수 있고, 날씨를 알려줄 수 있습니다.")

response = agent("오늘 날씨는 어떤가요?")
print(response)

### 도구 직접 호출하기

일부 애플리케이션에서는 도구를 직접 호출하는 것이 중요합니다. 예를 들어, 도구를 디버그하거나, 고객 정보로 에이전트 지식을 미리 채우거나, 다른 도구 내에서 도구를 사용하고 싶을 수 있습니다. Strands에서는 에이전트의 ``tool`` 메서드 다음에 도구 이름을 사용하여 이를 수행할 수 있습니다.

In [ ]:
# 또는 다음과 같이 도구를 직접 호출할 수 있습니다:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)


### 3. 로그 레벨과 형식 변경하기:

Strands SDK는 Python의 표준 `logging` 모듈을 사용하여 작업에 대한 가시성을 제공합니다.

Strands Agents SDK는 간단한 로깅 접근 방식을 구현합니다:

1. **모듈 레벨 로거**: SDK의 각 모듈은 logging.getLogger(__name__)을 사용하여 자체 로거를 만들며, 계층적 로깅을 위한 Python 모범 사례를 따릅니다.
2. **루트 로거**: SDK의 모든 로거는 "strands" 루트 로거의 자식이므로 전체 SDK에 대한 로깅을 쉽게 구성할 수 있습니다.
3. **기본 동작**: 기본적으로 SDK는 핸들러나 로그 레벨을 구성하지 않으므로 애플리케이션의 로깅 구성과 통합할 수 있습니다.

Strands Agents SDK에 대한 로깅을 활성화하려면 **"strands"** 로거를 구성할 수 있습니다. 디버깅 중에 로그 레벨을 변경하거나 로그 형식을 수정하려면 다음과 같이 로거 구성을 설정할 수 있습니다:

In [ ]:
import logging
from strands import Agent

# Strands 디버그 로그 레벨 활성화
logging.getLogger("strands").setLevel(logging.DEBUG) # 또는 logging.INFO

# 로깅 형식을 설정하고 로그를 stderr로 스트림
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")  # 선택사항: 모델 ID 지정
agent("안녕하세요!")


### 4. 모델 제공자

기본 모델 제공자는 [Amazon Bedrock](https://aws.amazon.com/bedrock/)이고 기본 모델은 현재 AWS 환경 지역의 Claude 3.7 Sonnet입니다.

모델 ID 문자열을 직접 제공하여 Amazon Bedrock에서 다른 모델을 지정할 수 있습니다:

In [ ]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)


모델 구성에 대한 더 많은 제어를 위해 `BedrockModel` 제공자 인스턴스를 만들 수 있습니다:

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# BedrockModel 만들기
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


사용 가능한 모델 제공자에 대한 자세한 내용은 [모델 제공자 빠른 시작 페이지](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)를 참조하세요.


**축하합니다!! 이제 Strands를 사용하여 간단한 에이전트를 구축하는 방법을 배웠습니다!!**

## [선택사항] 작업별 에이전트 구축하기 - RecipeBot 🍽️

작업별 에이전트의 실용적인 예제를 만들어보겠습니다. 레시피를 추천하고 요리 관련 질문에 답하는 `RecipeBot`을 만들어보겠습니다. 시작해봅시다!!

우리가 만들 것은 다음과 같습니다:

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [ ]:
# 필요한 패키지 설치
%pip install ddgs # 아직 설치하지 않았다면 strands-agents strands-agents-tools도 설치하세요

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# 로깅 구성
logging.getLogger("strands").setLevel(logging.INFO)

# 웹 검색 도구 정의
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """웹을 검색하여 최신 정보를 가져옵니다.
    Args:
        keywords (str): 검색 쿼리 키워드.
        region (str): 검색 지역: wt-wt, us-en, uk-en, ru-ru 등..
        max_results (int | None): 반환할 최대 결과 수.
    Returns:
        검색 결과가 포함된 딕셔너리 목록.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "결과를 찾을 수 없습니다."
    except RatelimitException:
        return "RatelimitException: 잠시 후 다시 시도해주세요."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [ ]:
# 레시피 어시스턴트 에이전트 만들기
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    system_prompt="""당신은 도움이 되는 요리 어시스턴트인 RecipeBot입니다.
    재료를 기반으로 레시피를 찾고 요리 질문에 답해주세요.
    사용자가 재료를 언급하거나 요리 정보를 찾을 때 웹 검색 도구를 사용하여
    레시피를 찾아주세요.""",
    # 위에서 만든 웹 검색 도구 가져오기
    tools=[websearch],
)

In [ ]:
response = recipe_agent("닭고기와 브로콜리를 사용한 레시피를 추천해주세요.")

# 다른 예시들:
# response = recipe_agent("퀴노아는 어떻게 요리하나요?")
# response = recipe_agent("새우 파스타에서 화이트 와인을 어떻게 대체할 수 있나요?")
# response = recipe_agent("아스파라거스의 건강상 이점은 무엇인가요?")
print(response)

#### 그게 다입니다! 이제 단 몇 줄의 코드로 도구를 가진 실행 중인 사용 사례 에이전트를 만들었습니다 🥳.

더 자세한 빠른 시작 가이드는 [Strands 문서](https://strandsagents.com/0.1.x/user-guide/quickstart/)를 확인하세요.

### [선택사항] CLI를 통해 RecipeBot 실행하기: 
`02_simple_interactive_usecase/recipe_bot.py`에 제공된 Python 스크립트를 통해 명령줄에서 대화형 모드로 에이전트를 실행할 수 있습니다(예: SageMaker Studio의 터미널 사용). 이를 통해 더 동적인 방식으로 에이전트와 상호작용하고, CLI를 통해 메시지를 보내고 응답을 받을 수 있습니다.
명령줄 인터페이스에서 다음 명령을 실행하여 대화형 모드로 에이전트를 실행하세요:

```cli
cd samples/01-tutorials/01-fundamentals/01-first-agent/02-simple-interactive-usecase/
pip install -r requirements.txt
python recipe_bot.py
```

이를 통해 명령줄 인터페이스(CLI)를 통해 봇과 대화할 수 있습니다.